<a href="https://colab.research.google.com/github/yvrjsharma/HugginFace_Gradio/blob/main/DeepSeek_VL_Gradio_Multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepSeek-VL with Gradio Multimodal Chatbots

## Install gradio

In [1]:
!pip install gradio -q
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.3 MB/s eta 0:00:00


## Clone DeepSeek-VL lib

In [ ]:
!git clone https://github.com/deepseek-ai/DeepSeek-VL
%cd DeepSeek-VL

!pip install -e .

In [3]:
#no
!pip install typing-extensions

## Import libraries

In [5]:
import collections.abc
import sys
import torch
from transformers import AutoModelForCausalLM

# Temporarily fix the ImportError for collections.Mapping
sys.modules['collections.Mapping'] = collections.abc.Mapping
sys.modules['collections.MutableMapping'] = collections.abc.MutableMapping
sys.modules['collections.Sequence'] = collections.abc.Sequence

# Now try importing the deepseek_vl package
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images


/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Download and initialize 1.3b VLM chat model

In [7]:
# specify the path to the model
model_path = "deepseek-ai/deepseek-vl-1.3b-chat"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Chat Prompt

In [8]:
conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>Describe each stage of this image.",
        "images": ["/content/DeepSeek-VL/images/training_pipelines.png"]
    },
    {
        "role": "Assistant",
        "content": ""
    }
]

### Define graio predict function for chat inference

In [10]:

def deepseekvl(messages):

  pil_images = load_pil_images(messages)
  prepare_inputs = vl_chat_processor(
      conversations=messages,
      images=pil_images,
      force_batchify=True
  ).to(vl_gpt.device)
  # run image encoder to get the image embeddings
  inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)
  # run the model to get the response
  outputs = vl_gpt.language_model.generate(
      inputs_embeds=inputs_embeds,
      attention_mask=prepare_inputs.attention_mask,
      pad_token_id=tokenizer.eos_token_id,
      bos_token_id=tokenizer.bos_token_id,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=512,
      do_sample=False,
      use_cache=True)

  answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
  return answer


def bot(msg, history):

    messages = []
    history = history + [[msg, None]]
    history_len = len(history)
    image_flag=False
    for idx, (user_message, assistant_message) in enumerate(history):

      if idx == (history_len-1):
        break
      if image_flag:
        image_flag=False
        continue
      if isinstance(user_message, tuple):
        if idx == (history_len-2):
          messages.append({"role": "user",
                          "content": f"<image_placeholder>{msg}",
                          "images": [user_message[0]],})
          messages.append({"role": "Assistant", "content": ""})
          image_flag=True
          response = deepseekvl(messages)
          history[-1][1] = response
          return history, ''
        else:
          messages.append({"role": "user",
                           "content": f"<image_placeholder>{history[idx+1][0]}",
                           "images": [user_message[0]],})
          image_flag=True
      else:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})

    messages.append({"role": "user", "content": msg})
    messages.append({"role": "Assistant", "content": ""})
    response = deepseekvl(messages)
    history[-1][1] = response
    return history, ''


### Run this only if you get the below error while running inference in Gradio chatbot:

> RuntimeError: cutlassF: no kernel found to launch

Restart the session after this

In [ ]:
!pip install torch --upgrade

## **Gradio layout**

In [11]:
import gradio as gr
import os
import time

# Chatbot demo with multimodal input

def print_like_dislike(x: gr.LikeData):
    # for now just print the messages liked or disliked by a user
    # you can implement a more complex workflow here as well
    print(x.index, x.value, x.liked)

# upload file to chatbot
def add_file(history, file):
    history = history + [[(file.name,), None]]
    return history


with gr.Blocks(fill_height=True) as demo:
    gr.HTML("""<h1><center> DeepSeek-VL (1.3B Parameters) </center></h1>""")
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        #avatar_images=(None, ("/content/deepseeklogo.png")),
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="Enter text and press enter, or upload an image",
            container=False,
        )
        btn = gr.UploadButton("📁", file_types=["image", "video", "audio"])

    txt_msg = txt.submit(bot, [txt, chatbot], [chatbot,txt] )

    txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False)

    chatbot.like(print_like_dislike, None, None)


demo.queue()
demo.launch(debug=True,)



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c07cb48b1f0ba7903b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c07cb48b1f0ba7903b.gradio.live


In [ ]:
9